In [1]:
import numpy as np
from Tomita_Grammars import tomita_3
from Training_Functions import make_train_set_for_target
import tensorflow as tf

print(f'Tensorflow: {tf.__version__}')


Tensorflow: 2.10.0


In [2]:
target = tomita_3
alphabet = "01"

In [3]:
train_set = make_train_set_for_target(target,alphabet)

print(len(train_set))
print(list(train_set.items())[:10])

made train set of size: 2772 , of which positive examples: 1326
2772
[('', True), ('0', True), ('1', True), ('11', True), ('01', True), ('00', True), ('10', False), ('000', True), ('100', True), ('111', True)]


In [4]:
words = list(train_set.keys())
pre_target = list(train_set.values())
print(words[:10])
print(pre_target[:10])

target = []
for i in pre_target:
    if i:
        target.append([0,1])
    else:
        target.append([1,0])
print(target[:10])


['', '0', '1', '11', '01', '00', '10', '000', '100', '111']
[True, True, True, True, True, True, False, True, True, True]
[[0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [1, 0], [0, 1], [0, 1], [0, 1]]


In [5]:
char_set = ['<pad>'] + list(alphabet)
idx2char = {idx:char for idx, char in enumerate(char_set)}
char2idx = {char:idx for idx, char in enumerate(char_set)}
char2idx

{'<pad>': 0, '0': 1, '1': 2}

In [6]:
X = list(map(lambda word: [char2idx.get(char) for char in word], words))
X_len = list(map(lambda word: len(word), X))
X_target = list(zip(X, target))
np.random.shuffle(X_target)
X = [i[0] for i in X_target]
target = np.array([j[1] for j in X_target])


In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_sequence = 30

X = pad_sequences(X, maxlen=max_sequence, padding='post', truncating='post')

X_train, X_val, target_train, target_val = X[:2100], X[2100:], target[:2100], target[2100:]


In [8]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, target_train)).shuffle(buffer_size=4).batch(batch_size=32)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, target_val)).batch(batch_size=50)
train_ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 30), dtype=tf.int32, name=None), TensorSpec(shape=(None, 2), dtype=tf.int32, name=None))>

In [9]:
input_dim = len(char2idx)
output_dim = len(char2idx)

In [10]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers

model = Sequential()
model.add(layers.Embedding(input_dim=input_dim, output_dim=output_dim, mask_zero=True,
             input_length=max_sequence, trainable=False, 
             embeddings_initializer=tf.keras.initializers.random_normal()))
model.add(layers.SimpleRNN(units=10, activation='sigmoid', 
                          kernel_initializer='random_uniform'))
model.add(layers.Dense(2, activation='sigmoid'))


    



model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 3)             9         
                                                                 
 simple_rnn (SimpleRNN)      (None, 10)                140       
                                                                 
 dense (Dense)               (None, 2)                 22        
                                                                 
Total params: 171
Trainable params: 162
Non-trainable params: 9
_________________________________________________________________


In [11]:
loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.005)
# optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.005)
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)

In [12]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.CategoricalAccuracy(name='val_accuracy')

In [13]:
@tf.function
def train_step(X, target):
    with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
        predictions = model(X, training=True)
        loss = loss_object(target, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(target, predictions)

In [14]:
@tf.function
def val_step(X_val, target_val):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(X_val, training=False)
    loss = loss_object(target_val, predictions)
    val_loss(loss)
    val_accuracy(target_val, predictions)

In [ ]:
EPOCHS = 500

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    val_loss.reset_states()
    val_accuracy.reset_states()

    for X_train, target_train in train_ds:
        train_step(X_train, target_train)

    for X_val, target_val in val_ds:
        val_step(X_val, target_val)
    if (epoch + 1) % 100 == 0:

        print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Acc: {train_accuracy.result() * 100}, '
        f'val Loss: {val_loss.result()}, '
        f'val Acc: {val_accuracy.result() * 100}'
        )

Epoch 100, Loss: 0.05968109518289566, Acc: 93.0952377319336, val Loss: 0.05841085687279701, val Acc: 92.55952453613281
Epoch 200, Loss: 0.034465767443180084, Acc: 95.95237731933594, val Loss: 0.046292949467897415, val Acc: 93.60118865966797
Epoch 300, Loss: 0.010198348201811314, Acc: 99.28571319580078, val Loss: 0.01529709342867136, val Acc: 98.66071319580078
Epoch 400, Loss: 0.0006343844579532743, Acc: 100.0, val Loss: 0.0026446322444826365, val Acc: 99.85118865966797
